In [1]:
import pyperclip as clip

In [2]:
races = ['BL', 'WH', 'AM', 'AS', 'HI', 'HP', 'TR']
genders = ['M', 'F', 'both_genders']
disciplinary_metrics = ['MULTOOS', 'SINGOOS', 'REF', 'ARR']
academic_metrics = ['GTENR', 'APENR']
disability = ['WODIS', 'WDIS']

### District rate calculations

In [3]:
dist_query_segments = 'SELECT "LEAID"'

In [4]:
for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
        for race in races:
            for gender in genders:
                if gender == 'both_genders':
                    district_query = f'\n, ("SCH_DISC{dis_status}_{metric}_{race}_F" + "SCH_DISC{dis_status}_{metric}_{race}_M") / (NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0)) as {race}_{gender}_{dis_status}_{metric}_rate'

                else:
                    district_query = f'\n, ("SCH_DISC{dis_status}_{metric}_{race}_{gender}" / NULLIF("SCH_ENR_{race}_{gender}", 0)) as {race}_{gender}_{dis_status}_{metric}_rate'

                dist_query_segments += district_query

In [5]:
for metric in academic_metrics:
    for race in races:
        for gender in genders:
            if gender == 'both_genders':
                district_query = f'\n, ("SCH_{metric}_{race}_F" + "SCH_{metric}_{race}_M") / (NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0)) as {race}_{gender}_{metric}_rate'

            else:
                district_query = f'\n, ("SCH_{metric}_{race}_{gender}" / NULLIF("SCH_ENR_{race}_{gender}", 0)) as {race}_{gender}_{metric}_rate'

            dist_query_segments += district_query

In [6]:
dist_query_segments += ' from districts;'
print(dist_query_segments)

SELECT "LEAID"
, ("SCH_DISCWODIS_MULTOOS_BL_M" / NULLIF("SCH_ENR_BL_M", 0)) as BL_M_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_BL_F" / NULLIF("SCH_ENR_BL_F", 0)) as BL_F_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_BL_F" + "SCH_DISCWODIS_MULTOOS_BL_M") / (NULLIF("SCH_ENR_BL_F", 0) + NULLIF("SCH_ENR_BL_M", 0)) as BL_both_genders_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_WH_M" / NULLIF("SCH_ENR_WH_M", 0)) as WH_M_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_WH_F" / NULLIF("SCH_ENR_WH_F", 0)) as WH_F_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_WH_F" + "SCH_DISCWODIS_MULTOOS_WH_M") / (NULLIF("SCH_ENR_WH_F", 0) + NULLIF("SCH_ENR_WH_M", 0)) as WH_both_genders_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_AM_M" / NULLIF("SCH_ENR_AM_M", 0)) as AM_M_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_AM_F" / NULLIF("SCH_ENR_AM_F", 0)) as AM_F_WODIS_MULTOOS_rate
, ("SCH_DISCWODIS_MULTOOS_AM_F" + "SCH_DISCWODIS_MULTOOS_AM_M") / (NULLIF("SCH_ENR_AM_F", 0) + NULLIF("SCH_ENR_AM_M", 0)) as AM_both_gend

In [7]:
clip.copy(dist_query_segments)

### National Rate calculations

Calculate national rate for:
1. White student groups for each metric
2. All student groups for each metric

In [83]:
national_query = 'Select 1'

In [84]:
race = 'WH'
for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
        for gender in genders:
            if gender == 'both_genders':
                query_segment = f'\n, SUM("SCH_DISC{dis_status}_{metric}_{race}_F" + "SCH_DISC{dis_status}_{metric}_{race}_M") / SUM((NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0))) as ntl_{race}_{gender}_{dis_status}_{metric}_rate'

            else:
                query_segment = f'\n, SUM("SCH_DISC{dis_status}_{metric}_{race}_{gender}") / SUM(NULLIF("SCH_ENR_{race}_{gender}", 0)) as ntl_{race}_{gender}_{dis_status}_{metric}_rate'

            national_query += query_segment
            
for metric in academic_metrics:
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("SCH_{metric}_{race}_F" + "SCH_{metric}_{race}_M") / SUM(NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0)) as ntl_{race}_{gender}_{metric}_rate'

        else:
            query_segment = f'\n, SUM("SCH_{metric}_{race}_{gender}") / SUM(NULLIF("SCH_ENR_{race}_{gender}", 0)) as ntl_{race}_{gender}_{metric}_rate'

        national_query += query_segment

In [85]:
national_query += ' from districts;'

In [86]:
clip.copy(national_query)

Sums across all races

In [79]:
all_races_qry = 'SELECT 1'

for metric in disciplinary_metrics:
    for dis_status in disability:
        if dis_status == 'WDIS':
            metric += '_IDEA'
        for gender in genders:
            if gender == 'both_genders':
                query_segment = f'\n, SUM("TOT_DISC{dis_status}_{metric}_F" + "TOT_DISC{dis_status}_{metric}_M") / SUM((NULLIF("TOT_ENR_F", 0) + NULLIF("TOT_ENR_M", 0))) as ntl_all_races_{gender}_{dis_status}_{metric}_rate'

            else:
                query_segment = f'\n, SUM("TOT_DISC{dis_status}_{metric}_{gender}") / SUM(NULLIF("TOT_ENR_{gender}", 0)) as ntl_all_races_{gender}_{dis_status}_{metric}_rate'

            all_races_qry += query_segment
            
for metric in academic_metrics:
    for gender in genders:
        if gender == 'both_genders':
            query_segment = f'\n, SUM("TOT_{metric}_F" + "TOT_{metric}_M") / SUM(NULLIF("TOT_ENR_F", 0) + NULLIF("TOT_ENR_M", 0)) as ntl_all_races_{gender}_{metric}_rate'

        else:
            query_segment = f'\n, SUM("TOT_{metric}_{gender}") / SUM(NULLIF("TOT_ENR_{gender}", 0)) as ntl_all_races_{gender}_{metric}_rate'

        all_races_qry += query_segment
        
all_races_qry += ' from districts;'        

In [82]:
print(all_races_qry)

SELECT 1
, SUM("TOT_DISCWODIS_MULTOOS_M") / SUM(NULLIF("TOT_ENR_M", 0)) as ntl_all_races_M_WODIS_MULTOOS_rate
, SUM("TOT_DISCWODIS_MULTOOS_F") / SUM(NULLIF("TOT_ENR_F", 0)) as ntl_all_races_F_WODIS_MULTOOS_rate
, SUM("TOT_DISCWODIS_MULTOOS_F" + "TOT_DISCWODIS_MULTOOS_M") / SUM((NULLIF("TOT_ENR_F", 0) + NULLIF("TOT_ENR_M", 0))) as ntl_all_races_both_genders_WODIS_MULTOOS_rate
, SUM("TOT_DISCWDIS_MULTOOS_IDEA_M") / SUM(NULLIF("TOT_ENR_M", 0)) as ntl_all_races_M_WDIS_MULTOOS_IDEA_rate
, SUM("TOT_DISCWDIS_MULTOOS_IDEA_F") / SUM(NULLIF("TOT_ENR_F", 0)) as ntl_all_races_F_WDIS_MULTOOS_IDEA_rate
, SUM("TOT_DISCWDIS_MULTOOS_IDEA_F" + "TOT_DISCWDIS_MULTOOS_IDEA_M") / SUM((NULLIF("TOT_ENR_F", 0) + NULLIF("TOT_ENR_M", 0))) as ntl_all_races_both_genders_WDIS_MULTOOS_IDEA_rate
, SUM("TOT_DISCWODIS_SINGOOS_M") / SUM(NULLIF("TOT_ENR_M", 0)) as ntl_all_races_M_WODIS_SINGOOS_rate
, SUM("TOT_DISCWODIS_SINGOOS_F") / SUM(NULLIF("TOT_ENR_F", 0)) as ntl_all_races_F_WODIS_SINGOOS_rate
, SUM("TOT_DISCWODIS_SI

In [81]:
clip.copy(all_races_qry)